In [58]:
import matplotlib.pyplot as plt
import numpy as np
import IPython
from scipy import signal
from scipy.io import wavfile
from ipywidgets import interact, fixed, IntSlider, HBox, Layout, Output, VBox
import ipywidgets as widgets

%matplotlib widget

# Experimentation

In [175]:
SF, s = wavfile.read('audiofile.wav')
# s = s[::10]
# SF = SF//10
IPython.display.Audio(s, rate=SF)
SF, len(s)

(44100, 882151)

In [176]:
plt.close('all')
plt.figure(figsize=(8, 2))
plt.plot(np.linspace(0, len(s)/SF, len(s)), s, linewidth=0.1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [177]:
s_FT = np.abs(np.fft.fftshift(np.fft.fft(s)))
s_FT = s_FT / s_FT.max()
w_FT = np.linspace(-SF//2, SF//2, len(s))

plt.close('all')
plt.figure(figsize=(8, 4))
plt.plot(w_FT[3*len(w_FT)//8:5*len(w_FT)//8], s_FT[3*len(s_FT)//8:5*len(s_FT)//8])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [221]:
# Constructing the filter
order = 5
f_crit = [1000, 5000]
b, a = signal.butter(N=order, Wn=f_crit, btype='bandstop', fs=SF)
# Frequency response
w, h = signal.freqz(b, a, whole=True, fs=SF)
h = np.abs(np.fft.fftshift(h))
w -= SF//2
# Filtering
s_filtered = signal.lfilter(b, a, s)

In [224]:
plt.close('all')
plt.figure(figsize=(10,5))
q = 10
plt.subplot(121)
# plt.plot(w_FT[(q//2-1)*len(w_FT)//q:(q//2+1)*len(w_FT)//q], s_FT[(q//2-1)*len(s_FT)//q:(q//2+1)*len(s_FT)//q], linewidth=0.2)
# plt.plot(w[(q//2-1)*len(w)//q:(q//2+1)*len(w)//q], h[(q//2-1)*len(w)//q:(q//2+1)*len(w)//q])
plt.plot(w_FT, s_FT, linewidth=0.2)
plt.plot(w, h)
plt.subplot(122)
s_FT_filt = np.abs(np.fft.fftshift(np.fft.fft(s_filtered)))
# plt.plot(w_FT[(q//2-1)*len(w_FT)//q:(q//2+1)*len(w_FT)//q], s_FT_filt[(q//2-1)*len(s_FT)//q:(q//2+1)*len(s_FT)//q], linewidth=0.2)
plt.plot(w_FT, s_FT_filt, linewidth=0.2)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [204]:
plt.close('all')
plt.figure(figsize=(8, 4))
plt.subplot(211)
plt.plot(np.linspace(0, len(s)/SF, len(s)), s, linewidth=0.1)
plt.subplot(212)
plt.plot(np.linspace(0, len(s)/SF, len(s)), s_filtered, linewidth=0.1)
plt.show()
out1 = Output(layout={'width': '320px', 'height': '60px'})
out2 = Output(layout={'width': '320px', 'height': '60px'})
out1.append_display_data(IPython.display.Audio(s, rate=SF))
out2.append_display_data(IPython.display.Audio(s_filtered, rate=SF))

text1 = widgets.HTML(value="<h3>Original</h3>")
text2 = widgets.HTML(value="<h3>Filtered</h3>")

display(VBox([HBox([text1, out1]), HBox([text2, out2])]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Class definition

In [71]:
class Filter_Demo():
    def __init__(self, filename):
        self.out = Output(layout={'width': '980px', 'height': '450px'})
        self.axs = []
        
        # Read the audio signal from file
        self.SF, self.s = wavfile.read(filename)
        self.t = np.linspace(0, len(self.s)/self.SF, len(self.s))
        
        # Generate Fourier Transform of audio signal
        s_FT = np.abs(np.fft.fftshift(np.fft.fft(self.s)))
        self.s_FT = s_FT / s_FT.max()
        self.w_FT = np.linspace(-self.SF//2, self.SF//2, len(self.s))
        
        # Filter types
        self.filter_types = ['lowpass', 'highpass', 'bandpass', 'bandstop']
        
        self.f_crit = self.SF//6
        self.filter = self.filter_types[0]
        self.s_filtered = None
        self.h = None
        self.w = None

        # Compute the initial filter
        self.update_filter()
        
        # Inizializate the figure
        self.init_figure()
        
        # Add audio players
        self.play_orig = Output(layout={'width': '320px', 'height': '60px'})
        self.play_filt = Output(layout={'width': '320px', 'height': '60px'})
        self.play_orig.append_display_data(IPython.display.Audio(self.s, rate=self.SF))
        self.play_filt.append_display_data(IPython.display.Audio(self.s_filtered, rate=self.SF))
        # Descriptive text
        self.text_orig = widgets.HTML(value="<h3>Original</h3>")
        self.text_filt = widgets.HTML(value="<h3>Filtered</h3>")

        # Add frequency sliders
        self.f0_slider = widgets.IntSlider(value=self.SF//6, min=0, max=self.SF//2, description='$f_0$:')
#         self.phase_check.observe(self.show_phase_callback, names='value')
        self.f1_slider = widgets.IntSlider(value=2*self.SF//6, min=0, max=self.SF//2, description='$f_1$:')
        
        # Add dropdown menu
        self.filter_menu = widgets.Dropdown(options=self.filter_types, value=self.filter_types[0], 
                                            description='Filter type:', layout={'width': 'max-content'})
        
        display(VBox([self.out, HBox([self.filter_menu, VBox([self.f0_slider, self.f1_slider]), VBox([HBox([self.text_orig, self.play_orig]), HBox([self.text_filt, self.play_filt])])])]))
        plt.tight_layout(pad=0.1, w_pad=1.0, h_pad=0.1)
        
    
    def init_figure(self):
        with self.out:
            self.fig = plt.figure(figsize=(8.5, 4))
            self.gs = self.fig.add_gridspec(2, 2)

            # Plot the FT
            self.axs.append(self.fig.add_subplot(self.gs[:, 0]))
            self.axs[0].set_title("Filter and signal spectrum")
            self.axs[0].plot(self.w_FT, self.s_FT, color='blue', linewidth=0.2)
            self.axs[0].plot(self.w, self.h, color='lightsalmon', linewidth=0.5)
            self.axs[0].set_xlabel('f [Hz]')
            self.axs[0].fill(self.w, self.h, facecolor='lightsalmon')
            self.axs[0].legend(['Signal', 'Filter'], loc='upper right')
            
            # Plot the original waveform
            self.axs.append(self.fig.add_subplot(self.gs[0, -1]))
            self.axs[1].set_title('Original signal')
            self.axs[1].plot(self.t, self.s, color='blue', linewidth=0.2)
            self.axs[1].set_xlabel('t [s]')
            self.axs[1].set_xlim([np.min(self.t), np.max(self.t)])
            self.axs[1].set_ylim([np.min(self.s), np.max(self.s)])
            self.axs[1].get_yaxis().set_visible(False)
            
            # Plot the filtered waveform
            self.axs.append(self.fig.add_subplot(self.gs[1, -1]))
            self.axs[2].set_title("Filtered signal")
            self.axs[2].plot(self.t, self.s_filtered, color='blue', linewidth=0.2)
            self.axs[2].set_xlabel('t [s]')
            self.axs[2].set_xlim([np.min(self.t), np.max(self.t)])
            self.axs[2].set_ylim([np.min(self.s), np.max(self.s)])
            self.axs[2].get_yaxis().set_visible(False)

    def update_filter(self):
        # Constructing the filter
        b, a = signal.butter(N=5, Wn=self.f_crit, btype=self.filter, fs=self.SF)
        # Frequency response
        w, h = signal.freqz(b, a, whole=True, fs=self.SF)
        self.h = np.abs(np.fft.fftshift(h))
        self.w = w - self.SF//2
        # Filtering
        self.s_filtered = signal.lfilter(b, a, self.s)



In [72]:
plt.close('all')
fd = Filter_Demo('audiofile.wav')